In [3]:
import pandas as pd

# Load raw transactions
transactions_raw = pd.read_excel("QVI_transaction_data.xlsx", engine="openpyxl")


# Work on a copy
transactions = transactions_raw.copy()

# Fix DATE
transactions["DATE"] = pd.to_datetime(transactions["DATE"], origin="1899-12-30", unit="D")

# Remove non-chip products (e.g., salsa)
transactions = transactions[~transactions["PROD_NAME"].str.contains("salsa", case=False)]

# Extract PACK_SIZE
transactions["PACK_SIZE"] = transactions["PROD_NAME"].str.extract(r"(\d+)")
transactions["PACK_SIZE"] = transactions["PACK_SIZE"].astype(int)

# Remove outliers (e.g., pack sizes < 50g or > 400g)
transactions = transactions[(transactions["PACK_SIZE"] >= 50) & (transactions["PACK_SIZE"] <= 400)]

# Extract BRAND (first word of product name)
transactions["BRAND"] = transactions["PROD_NAME"].str.split().str[0]

# Save cleaned transactions
transactions.to_csv("transactions_clean.csv", index=False)

In [4]:
print("transactions_clean.csv created successfully with shape:", transactions.shape)


transactions_clean.csv created successfully with shape: (246742, 10)


In [5]:
import pandas as pd

# -------------------------
# 1. Load raw customer data (CSV)
# -------------------------
customers_raw = pd.read_csv("QVI_purchase_behaviour.csv")

# Work on a copy
customers = customers_raw.copy()

# -------------------------
# 2. Standardize column names
# -------------------------
customers.columns = customers.columns.str.strip().str.upper()

# -------------------------
# 3. Quick checks
# -------------------------
print(customers.info())
print(customers.head())

# -------------------------
# 4. Save cleaned customers
# -------------------------
customers.to_csv("customers_clean.csv", index=False)

print("customers_clean.csv created successfully with shape:", customers.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72637 entries, 0 to 72636
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   LYLTY_CARD_NBR    72637 non-null  int64 
 1   LIFESTAGE         72637 non-null  object
 2   PREMIUM_CUSTOMER  72637 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB
None
   LYLTY_CARD_NBR               LIFESTAGE PREMIUM_CUSTOMER
0            1000   YOUNG SINGLES/COUPLES          Premium
1            1002   YOUNG SINGLES/COUPLES       Mainstream
2            1003          YOUNG FAMILIES           Budget
3            1004   OLDER SINGLES/COUPLES       Mainstream
4            1005  MIDAGE SINGLES/COUPLES       Mainstream
customers_clean.csv created successfully with shape: (72637, 3)


In [6]:
import pandas as pd

# Load cleaned files
transactions = pd.read_csv("transactions_clean.csv")
customers = pd.read_csv("customers_clean.csv")

# -------------------------
# 1. Merge datasets
# -------------------------
chips_joined = transactions.merge(customers, on="LYLTY_CARD_NBR", how="left")

# -------------------------
# 2. Add derived fields
# -------------------------

# Promo flag (example: if product name contains 'promo')
chips_joined["PROMO_FLAG"] = chips_joined["PROD_NAME"].str.contains("promo", case=False, na=False)

# Week column (start of week for each transaction)
chips_joined["WEEK"] = pd.to_datetime(chips_joined["DATE"]).dt.to_period("W").apply(lambda r: r.start_time)

# -------------------------
# 3. Save joined dataset
# -------------------------
chips_joined.to_csv("chips_joined.csv", index=False)

print("chips_joined.csv created successfully with shape:", chips_joined.shape)
print(chips_joined.head())

chips_joined.csv created successfully with shape: (246742, 14)
         DATE  STORE_NBR  LYLTY_CARD_NBR  TXN_ID  PROD_NBR  \
0  2018-10-17          1            1000       1         5   
1  2019-05-14          1            1307     348        66   
2  2019-05-20          1            1343     383        61   
3  2018-08-17          2            2373     974        69   
4  2018-08-18          2            2426    1038       108   

                                  PROD_NAME  PROD_QTY  TOT_SALES  PACK_SIZE  \
0    Natural Chip        Compny SeaSalt175g         2        6.0        175   
1                  CCs Nacho Cheese    175g         3        6.3        175   
2    Smiths Crinkle Cut  Chips Chicken 170g         2        2.9        170   
3    Smiths Chip Thinly  S/Cream&Onion 175g         5       15.0        175   
4  Kettle Tortilla ChpsHny&Jlpno Chili 150g         3       13.8        150   

     BRAND               LIFESTAGE PREMIUM_CUSTOMER  PROMO_FLAG       WEEK  
0  Natural  